# Navigation

In [1]:
import rospy
import tf
import math
import time
import actionlib
import numpy as np
from math import radians, degrees


from actionlib_msgs.msg import *
from std_msgs.msg import Float32, Int32
from darknet_ros_msgs.msg import BoundingBoxes
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from geometry_msgs.msg import PoseWithCovarianceStamped, PoseStamped, Quaternion, Point, Twist
# Initializing a neavigation node
rospy.init_node('map_navi',anonymous = False)
# move_base action as 'ac'
ac = actionlib.SimpleActionClient('move_base', MoveBaseAction)
goal = MoveBaseGoal()
goal.target_pose.header.frame_id = 'map'
goal.target_pose.header.stamp = rospy.Time.now()

current_pose = PoseWithCovarianceStamped()
processing = False   

# callback function for navigation
def callback(msg):
    global processing, current_pose
    if not processing:
        processing = True
        current_pose = msg.pose.pose
        
# subscribing topic '/amcl_pose'
rospy.Subscriber('/amcl_pose', PoseWithCovarianceStamped, callback)


# a function that converts from Quaternion to Euler
convert_to_nparray = lambda o: np.array([o.x, o.y, o.z, o.w])
def convert_to_euler(pose):
    tmp = convert_to_nparray(pose.orientation)
    tmp = tf.transformations.euler_from_quaternion(tmp)
    return tmp

# a function that converts from Euler to Quaternion
convert_to_rad = lambda o: np.array(o)*math.pi/180.
def convert_to_quaternion(euler):
    p, q, r = convert_to_rad(euler)
    tmp = tf.transformations.quaternion_from_euler(p, q, r)
    return Quaternion(*tmp)


# functions that starts the navigation task with given coordinates
def goal_pose(goal_list):
    global goal
    goal.target_pose.pose.position.x = goal_list[0]
    goal.target_pose.pose.position.y = goal_list[1]
    goal.target_pose.pose.orientation = convert_to_quaternion([0,0,goal_list[2]])
    goal.target_pose.header.stamp = rospy.Time.now()
    ac.send_goal(goal)
    #ac.wait_for_result()

def goal_orientation(angle):
    global goal
    goal.target_pose.pose.orientation = convert_to_quaternion([0,0,angle])
    goal.target_pose.header.stamp = rospy.Time.now()
    ac.send_goal(goal)
    ac.wait_for_result()

In [ ]:
goal_list = [[0.85, -0.9, -90],[1.94, 0.2, 90]]
trigger = 0
logic = False

for current_goal in goal_list:
    goal_pose(current_goal)
    while not ac.wait_for_result():
        pass
        #logic = ac.wait_for_result()
        #processing_yolo = False
        #if (person_check()  == 'close') and (trigger == 0):
            #trigger += 1
            #ac.cancel_goal()
            #time.sleep(5)
            #goal_pose(current_goal)
    goal_orientation(90)

# YOLO

In [ ]:
processing_yolo = False
detection_result = []

# a function that gets data => detection_result
def callback_yolo(msg):
    global processing_yolo, detection_result
    if not processing_yolo:
        processing_yolo = True
        if len(msg.bounding_boxes) > 0:
            detection_result = msg.bounding_boxes
        else:
            detection_result = []
    
# subscribing topic '/darknet_ros/bounding_boxes'
sub_yolo = rospy.Subscriber('/darknet_ros/bounding_boxes', BoundingBoxes, callback_yolo)
# initializing a node
#rospy.init_node('gen_command', anonymous = False)

In [ ]:
# bottle - 39 , vase - 75
def person_check():
    global detection_result
    for each in detection_result:
        if (each.id == 39) or (each.id == 75):   # 0 for person
            size = (each.xmax - each.xmin) * (each.ymax - each.ymin)
            if size > 20000:
                return 'close'  # close
            else:
                return 'not close'  # far

### Main

### other functions

In [ ]:
current_status = GoalStatusArray()
processing_status = False
def callback_status(msg):
    global processing_status, current_status
    if not processing_status:
        processing_status = True
        current_status = msg

processing_status = False
rospy.Subscriber('/move_base/status', GoalStatusArray, callback_status)

In [ ]:
current_goal = []
processing_goal = True
def callback_goal(msg):
    global processing_goal, current_goal
    if not processing_goal:
        processing_goal = True
        current_goal = [msg.pose.position.x, msg.pose.position.y]
        #print msg.pose #current_goal
            
# subscribing topic '/darknet_ros/bounding_boxes'
sub_goal = rospy.Subscriber('/move_base/current_goal', PoseStamped, callback_goal)

In [ ]:
# a function that pauses for 5 seconds
def pause():
    global pub, speed
    speed.linear.x = 0.0
    current_time = time.time()
    while True:
        pub.publish(speed)
        if time.time() - current_time > 5:
            break
    speed.linear.x = 0.01

In [ ]:
while True:
    processing_yolo = False
    size = person_pixel_size()
    if check_size(size) == 1:
        stop_function()
        break
    time.sleep(1)

print '==============DONE=============='